From Launch to Legacy: Tracking NASA’s Spacecraft Lifecycles

In [28]:
import requests
import pandas as pd
import sqlite3

In [29]:
# Importing CSV into Dataframe
space_mission_df = pd.read_csv('../tin/Resources/space_missions.csv', encoding='latin1')
space_mission_df.columns = space_mission_df.columns.str.lower()
space_mission_df.head()

,company,location,date,time,rocket,mission,rocketstatus,price,missionstatus
0,RVSN USSR,"Site 1/5, Baikonur Cosmodrome, Kazakhstan",1957-10-04,19:28:00,Sputnik 8K71PS,Sputnik-1,Retired,NaN,Success
1,RVSN USSR,"Site 1/5, Baikonur Cosmodrome, Kazakhstan",1957-11-03,02:30:00,Sputnik 8K71PS,Sputnik-2,Retired,NaN,Success
2,US Navy,"LC-18A, Cape Canaveral AFS, Florida, USA",1957-12-06,16:44:00,Vanguard,Vanguard TV3,Retired,NaN,Failure
3,AMBA,"LC-26A, Cape Canaveral AFS, Florida, USA",1958-02-01,03:48:00,Juno I,Explorer 1,Retired,NaN,Success
4,US Navy,"LC-18A, Cape Canaveral AFS, Florida, USA",1958-02-05,07:33:00,Vanguard,Vanguard TV3BU,Retired,NaN,Failure


In [31]:
# Dropping Price Column
space_mission_df = space_mission_df.drop('price', axis='columns')
space_mission_df.head()

,company,location,date,time,rocket,mission,rocketstatus,missionstatus
0,RVSN USSR,"Site 1/5, Baikonur Cosmodrome, Kazakhstan",1957-10-04,19:28:00,Sputnik 8K71PS,Sputnik-1,Retired,Success
1,RVSN USSR,"Site 1/5, Baikonur Cosmodrome, Kazakhstan",1957-11-03,02:30:00,Sputnik 8K71PS,Sputnik-2,Retired,Success
2,US Navy,"LC-18A, Cape Canaveral AFS, Florida, USA",1957-12-06,16:44:00,Vanguard,Vanguard TV3,Retired,Failure
3,AMBA,"LC-26A, Cape Canaveral AFS, Florida, USA",1958-02-01,03:48:00,Juno I,Explorer 1,Retired,Success
4,US Navy,"LC-18A, Cape Canaveral AFS, Florida, USA",1958-02-05,07:33:00,Vanguard,Vanguard TV3BU,Retired,Failure


In [32]:
# Data import into SQLite
conn = sqlite3.connect('space_missions.db')
space_mission_df.to_sql('missions', conn, if_exists='replace')

4630

In [33]:
# Identifying all Companies in table

conn = sqlite3.connect('space_missions.db')
cursor = conn.cursor()

cursor.execute("SELECT DISTINCT company FROM missions")
unique_companies = cursor.fetchall()
print("Unique Companies:")
for company in unique_companies:
    print(company[0])
print(f"Total Companies: {len(unique_companies)}")


Unique Companies:
RVSN USSR
US Navy
AMBA
US Air Force
NASA
General Dynamics
Martin Marietta
Armée de l'Air
UT
ASI
OKB-586
CECLES
Yuzhmash
CASC
CNES
Roscosmos
ISAS
RAE
MHI
ISRO
ESA
Arianespace
IAI
Boeing
Douglas
Northrop
Lockheed
VKS RF
MITT
ILS
EER
AEB
SRC
KCST
Sea Launch
Kosmotras
Eurockot
CASIC
Starsem
SpaceX
ULA
Land Launch
ISA
KARI
JAXA
Khrunichev
Blue Origin
Sandia
ExPace
Rocket Lab
Exos
Landspace
OneSpace
i-Space
IRGC
Virgin Orbit
Astra
Galactic Energy
Virgin Galactic
GK LS
Firefly
CAS Space
Total Companies: 62


In [34]:
# NASA Only
cursor.execute("SELECT * FROM missions WHERE company = 'NASA'")
nasa_only = cursor.fetchall()

nasa_df = pd.DataFrame(nasa_only, columns=[desc[0] for desc in cursor.description])
nasa_df = nasa_df.drop('index', axis='columns')
nasa_df

,company,location,date,time,rocket,mission,rocketstatus,missionstatus
0,NASA,"SLC-17A, Cape Canaveral AFS, Florida, USA",1958-10-11,08:42:00,Thor-DM 18 Able I,Pioneer 1,Retired,Partial Failure
1,NASA,"SLC-17A, Cape Canaveral AFS, Florida, USA",1958-11-08,07:30:00,Thor-DM 18 Able I,Pioneer 2,Retired,Failure
2,NASA,"LC-14, Cape Canaveral AFS, Florida, USA",1959-09-09,08:19:00,Atlas-D Mercury,Big Joe 1,Retired,Partial Failure
3,NASA,"LC-14, Cape Canaveral AFS, Florida, USA",1960-07-29,13:13:00,Atlas-D Mercury,Mercury-Atlas 1 (MA-1),Retired,Failure
4,NASA,"LC-5, Cape Canaveral AFS, Florida, USA",1960-11-21,14:00:00,Mercury-Redstone,Mercury-Redstone 1 (MR-1),Retired,Failure
...,...,...,...,...,...,...,...,...
198,NASA,"LC-39A, Kennedy Space Center, Florida, USA",2010-04-05,10:21:00,Space Shuttle Discovery,STS-131,Retired,Success
199,NASA,"LC-39A, Kennedy Space Center, Florida, USA",2010-05-14,18:20:00,Space Shuttle Atlantis,STS-132,Retired,Success
200,NASA,"LC-39A, Kennedy Space Center, Florida, USA",2011-02-24,21:53:00,Space Shuttle Discovery,STS-133,Retired,Success
201,NASA,"LC-39A, Kennedy Space Center, Florida, USA",2011-05-16,12:56:00,Space Shuttle Endeavour,STS-134,Retired,Success


In [35]:
# US Govt and Private Sector
us_companies = [
    "NASA", "US Navy", "US Air Force", "SpaceX", "Blue Origin", "Boeing",
    "Northrop", "Lockheed", "ULA", "General Dynamics", "Martin Marietta"
]

query = f"SELECT * FROM missions WHERE company IN ({','.join(['?']*len(us_companies))})"
cursor.execute(query, us_companies)
us_only = cursor.fetchall()

usa_df = pd.DataFrame(us_only, columns=[desc[0] for desc in cursor.description])
usa_df = usa_df.drop('index', axis='columns')
usa_df


,company,location,date,time,rocket,mission,rocketstatus,missionstatus
0,US Navy,"LC-18A, Cape Canaveral AFS, Florida, USA",1957-12-06,16:44:00,Vanguard,Vanguard TV3,Retired,Failure
1,US Navy,"LC-18A, Cape Canaveral AFS, Florida, USA",1958-02-05,07:33:00,Vanguard,Vanguard TV3BU,Retired,Failure
2,US Navy,"LC-18A, Cape Canaveral AFS, Florida, USA",1958-03-17,12:15:00,Vanguard,Vanguard 1,Retired,Success
3,US Navy,"LC-18A, Cape Canaveral AFS, Florida, USA",1958-04-28,02:53:00,Vanguard,Vanguard TV5,Retired,Failure
4,US Navy,"LC-18A, Cape Canaveral AFS, Florida, USA",1958-05-28,03:46:00,Vanguard,Vanguard SLV-1,Retired,Failure
...,...,...,...,...,...,...,...,...
1399,SpaceX,"SLC-4E, Vandenberg SFB, California, USA",2022-07-11,01:39:00,Falcon 9 Block 5,Starlink Group 3-1,Active,Success
1400,SpaceX,"LC-39A, Kennedy Space Center, Florida, USA",2022-07-15,00:44:00,Falcon 9 Block 5,CRS SpX-25,Active,Success
1401,SpaceX,"SLC-40, Cape Canaveral SFS, Florida, USA",2022-07-17,14:20:00,Falcon 9 Block 5,Starlink Group 4-22,Active,Success
1402,SpaceX,"SLC-4E, Vandenberg SFB, California, USA",2022-07-22,17:39:00,Falcon 9 Block 5,Starlink Group 3-2,Active,Success


In [37]:
# Identifying Unique NASA Spacecraft
nasa_unique_rockets = nasa_df['rocket'].unique()
nasa_unique_rockets

array(['Thor-DM 18 Able I', 'Atlas-D Mercury', 'Mercury-Redstone',
       'Scout X-1', 'Blue Scout II', 'Scout X-2', 'Scout X-3',
       'Scout X-4', 'Saturn I', 'Titan II GLV', 'Saturn IB', 'Saturn V',
       'Space Shuttle Columbia', 'Space Shuttle Challenger',
       'Space Shuttle Discovery', 'Space Shuttle Atlantis',
       'Space Shuttle Endeavour', 'Ares 1-X'], dtype=object)

In [ ]:
from datetime import datetime

# Converting date column to datetime format
nasa_df['date'] = pd.to_datetime(nasa_df['date'])

rocket_lifetimes = nasa_df.groupby('rocket')